# <p style="color:dodgerblue">Video Generation in Bedrock</p>
<hr style="border:1px dotted; color:floralwhite">

# Requirements for this lab
*See <span style="color:gold">Appendix</span> at the bottom of this lab to install requirements for this lab*

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:cadetblue">

# <p style="color:cadetblue">Import libraries and set clients</p>
*If the following does not show the latest (installed or upgraded) boto3 (at least v1.29.6), restart the kernel*

In [ ]:
import boto3, json
import base64
print (boto3.__version__)

print ('Done! Move to the next cell ->')

In [ ]:
import random
import datetime

# region_name is important especially if you're aws configure is set to ap-southeast-2 where there is no Bedrock yet
# define common vars
# nova reel is currently only available in us-east-1 (as of Dec 2024)
myRegion='us-east-1'

bedrockRun = boto3.client(service_name='bedrock-runtime', region_name=myRegion)
bedrockChk = boto3.client(service_name='bedrock', region_name=myRegion)

# s3
s3 = boto3.client('s3', region_name=myRegion)

# bucket - MUST BE A UNIQUE NAME hence the rnd
myBucket='doit-bedrock-video-' + str(random.randint(0, 1000)) + '-' + str(random.randint(0, 1000))
print (myBucket)

print ('Done! Move to the next cell ->')

-  <span style="color:greenyellow">If you want to provide your own resource image(s)!<span> 
-  <span style="color:greenyellow">REMEMBER TO CHANGE THIS PATH TO YOUR RESOURCES IF LOCAL!<span> 
-  <span style="color:greenyellow">IF IN AWS JUPYTER MAKE SURE THE 2ND IS UNCOMMENTED<span> 

In [ ]:
# local client path for resources
myLocalPathForDataSources='/Users/simondavies/Documents/GitHub/labs/bedrock/quick/Resources/'
# jupypter notebook path if notebook is used in AWS for example
#myLocalPathForDataSources='/home/ec2-user/SageMaker/labs/bedrock/quick/Resources/'

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:cadetblue">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create S3 Bucket</p>
- the video creation is an asynchronous call, it will write the movie to a defined S3 bucket and key when done
- you can also upload an image and use that as a starting reference for the video 
- defaults used, will use sse-s3 encryption and block public access

In [ ]:
# create bucket
if (myRegion != 'us-east-1'):
    s3.create_bucket(
        Bucket=myBucket, CreateBucketConfiguration={"LocationConstraint": myRegion}
    )
else:
    s3.create_bucket(
        Bucket=myBucket
    )

# create a "folder" - really keys as S3 is flat
s3.put_object(Bucket=myBucket, Key="videos/")
s3.put_object(Bucket=myBucket, Key="images/")
s3Videouri = 's3://{}/videos/'.format(myBucket)
s3Imageuri = 's3://{}/images/'.format(myBucket)
print (s3Videouri)
print (s3Imageuri)

print ('Done! Move to the next cell ->')

# <p style="color:crimson">Create Methods</p>

In [ ]:
# upload an image to be used as the first frame of the video
# we dont have to upload this to s3, can just read direct from myLocalPathForDataSources of course
# but done for completeness in case this code ever runs in the cloud as part of an app rather than vscode
# and the image is required for future reference
def uploadFirstFrameImage(img):
    files = [
        {
            "s3key": 'images/{}'.format(img),
            "localpath": "{}{}".format(myLocalPathForDataSources, img),
        },
    ]
    for file in files:
        print("uploading from: {}".format(file["localpath"]))
        s3.upload_file(
            file["localpath"],
            myBucket,
            file["s3key"],
            ExtraArgs={"StorageClass": "STANDARD"},
        )
        print("uploaded to: {}/{}".format(myBucket, file["s3key"]))

    # note even though we just uploaded to S3, we just get it from the myLocalPathForDataSources
    # no reason other than convenience, change to the bucket if you prefer  
    input_image_path = "{}{}".format(myLocalPathForDataSources, img)
    with open(input_image_path, "rb") as f:
        input_image_bytes = f.read()
        input_image_base64 = base64.b64encode(input_image_bytes).decode("utf-8")

    return input_image_base64

In [ ]:
# create a video from text
def createVideoFromText(m, prompt, s, ff, bytes):
    # prep some vars
    body=''

    # which model are we calling
    match m:
        case "amazon.nova-reel-v1:0":
            # Amazon - Nova Reel
            # https://docs.aws.amazon.com/nova/latest/userguide/video-generation.html
            body = {
                    "taskType": "TEXT_VIDEO",
                    "textToVideoParams": {
                        "text": prompt
                        # you can provide a first frame image if needed, see docu for details
                    },
                    "videoGenerationConfig": {
                        "durationSeconds": 6,
                        "fps": 24,
                        "dimension": "1280x720",
                        "seed": s,
                    },
                }

            # do we have an image to use as a first frame, if so we need to turn it into a byte stream
            if ff:
                body["textToVideoParams"]["images"] = [
                    {
                        "format": "jpeg",
                        "source": {
                            "bytes": bytes
                        }
                    }
                ]

            try:
                # invoke the model
                invocation = bedrockRun.start_async_invoke(
                    modelId=m,
                    modelInput=body,
                    outputDataConfig={
                        "s3OutputDataConfig": {
                            "s3Uri": s3Videouri
                        }
                    }
                )

                # print the response JSON
                print("Response:")
                print(json.dumps(invocation, indent=2, default=str))
                print("In progress, each 6 seconds takes approx 3 minutes. Started: {}".format(datetime.datetime.now()))
                
            except Exception as e:
                message = e.response["Error"]["Message"]
                print(f"Error: {message}")

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:lightblue">

# <p style="color:lightblue">Invoke a model with a text prompt</p>
1. Specify properties
2. Call the method

In [ ]:
# set up the prompt and params - note each model will have different ranges for its params
prompt = "Two children sit on a bench in the forest. \
        The trees move behind them, and a very cute baby red Welsh dragon sticks her head out of the trees. \
        She climbs over the bench and sits between them, blowing a small flame from her mouth into the sky to say a \
        friendly hello. The children smile excitedly. \
        The dragon then jumps off the bench leaving the children, and flys away into the sky. \
        Camera follows the dragon as she flys away."
prompt='A superhero is dressed in a black and yellow costume that has a white starburst on its chest. \
        The superhero is flying in a restaurant kitchen above a grill where chicken meat is being cooked. \
        His eyes have laser beams pointed onto the chicken meat helping them to cook evenly. \
        Cinematic, photographic quality, hyper detailed, life like. \
        The camera follows him as he hovers above the grill.'


# which model do you want to use based on the match statement below
u = 1

# do you want to provide a source image for the generation to use as a reference for its first frame
# image must be exactly 1280x720
ff = False
bytes=0
if ff:
    # must be a .jpeg (or .png if you want to change code above to accommodate)
    bytes=uploadFirstFrameImage("IMG_8037.jpg")

# the model will write the result to your s3 bucket in the videos key (folder)
match u:
    case 1:
        # Amazon - Nova Reel
        # property ranges: https://docs.aws.amazon.com/nova/latest/userguide/video-gen-access.html
        # Takes approx 3 minutes per 6 second video
        m = "amazon.nova-reel-v1:0"
        s = random.randint(0, 2147483646)  # random seed of the image

print("Seed is (in case you like the result!): {}".format(s))
createVideoFromText(m, prompt, s, ff, bytes)

#### <span style="color:deeppink">you can run the following cell multiple times until the status is Completed</span>
- use the invocationArn as shown above

In [ ]:
response = bedrockRun.get_async_invoke(
    invocationArn="arn:aws:bedrock:us-east-1:546709318047:async-invoke/qjkhrw7zo7yu"
)

status = response["status"]
print(f"Status: {status}, Time Now: {datetime.datetime.now()}")

# <p style="color:crimson">Delete S3 Bucket</p>
- Remember to delete the bucket when done

In [ ]:
# delete s3 bucket
# NOTE WARNING - this will delete all objects in the bucket with NO prompt or confirmation
# NOTE this deletes EVERYTHING
s3r = boto3.resource('s3')
bucket = s3r.Bucket(myBucket)
bucket.objects.all().delete()

# delete the bucket
response = s3.delete_bucket(Bucket=myBucket)

<hr style="border:1px dotted;color:lightblue">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Install Requirements (macOS)</p>
# Requirements for this lab

#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.</p>

*Windows requirements will be similar, apart from Homebrew.*  
* python must be installed on your client, and be at least v3.8
  * 3.8 supports the latest release of boto3 which supports Bedrock  
* boto3 must be installed on your client, and be at least v1.33.9  
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html
* 
* Bedrock model access requested
  * Models used in this lab with access already requested are:
    * AI21 Labs - Jurassic-2 Ultra and Jurassic-2 Mid
    * Meta Llama 2 Chat 13B
### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1. Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html

In [ ]:
%%bash
aws sts get-caller-identity

### <p style="color:gold">4. Request Bedrock model access</p> 
*You must request access to the models required, you may need to provide use case details before you are able to request*  
*Make sure you request in the region you intend to use the models in, this lab is us-west-2*  
https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/modelaccess  

Models required in this lab:

* See code above for use of models and what access to request

#### Pricing
*Price is calculated based on properties including quality of the image.*  
https://aws.amazon.com/bedrock/pricing/  
https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-prepare.html

### <p style="color:gold">5. Image editor that can create masks</p> 

https://www.gimp.org/downloads/

<hr style="border:1px dotted;color:gold">